In [5]:
import sys, os
sys.path.insert(0, os.path.abspath('..'))

In [3]:
# from lib.pipeline import Pipeline
# import numpy as np
# import matplotlib.pyplot as plt
# import torch
# from torchdrug import utils, data
# from lib.disable_logger import DisableLogger
# GPU = 1

# device = f'cuda:{GPU}'
# pipeline = Pipeline(
#     model='lm-gearnet',
#     dataset='atpbind3d',
#     gpus=[GPU],
#     model_kwargs={
#         'gpu': GPU,
#         'gearnet_hidden_dim_size': 512,
#         'gearnet_hidden_dim_count': 4,
#         'bert_freeze': False,
#         'bert_freeze_layer_count': 29,
#     },
#     batch_size=16,
# )

get dataset atpbind3d
Split num:  [337, 41, 41]
train samples: 337, valid samples: 41, test samples: 41


In [4]:
# import pandas as pd
# df = pd.DataFrame()

# def create_pred_dataframe(pipeline, dataset, weights):
#     weights_loaded = [
#         torch.load(weight, map_location='cpu') for weight in weights
#     ]
#     df = pd.DataFrame()
#     pipeline.task.eval()
#     for protein_index, batch in enumerate(data.DataLoader(dataset, batch_size=1, shuffle=False)):
#         batch = utils.cuda(batch, device=device)
#         if protein_index % 10 == 9:
#             print(f'processing protein {protein_index + 1} / {len(dataset)}')
#         label = pipeline.task.target(batch)['label'].flatten()
        
#         new_data = {
#             'protein_index': protein_index,
#             'residue_index': list(range(len(label))),
#             'target': label.tolist(),
#         }
#         for i, weight in enumerate(weights_loaded):
#             pipeline.task.load_state_dict(utils.cuda(weight, device=device) , strict=False)
#             pred = pipeline.task.predict(batch).flatten()
#             assert(len(label) == len(pred))
#             new_data[f'pred_{i}'] = [round(t, 5) for t in pred.tolist()]
#         new_data = pd.DataFrame(new_data)
#         df = pd.concat([df, new_data])
    
#     return df

# state_dict_files_5_20 = [
#     'rus_5_20_0.6435.pth',
#     'rus_5_21_0.6211.pth',
#     'rus_5_22_0.6401.pth',
#     'rus_5_23_0.6570.pth',
#     'rus_5_24_0.6322.pth',
#     'rus_5_25_0.6327.pth',
#     'rus_5_26_0.6214.pth',
#     'rus_5_27_0.6361.pth',
#     'rus_5_28_0.6372.pth',
#     'rus_5_29_0.6229.pth',
# ]

# df_valid = create_pred_dataframe(pipeline, dataset=pipeline.valid_set, weights=state_dict_files_5_20)
# df_test = create_pred_dataframe(pipeline, dataset=pipeline.test_set, weights=state_dict_files_5_20)

# prefix = 'rus_5_20'
# df_valid.to_csv(f'{prefix}_valid.csv', index=False)
# df_test.to_csv(f'{prefix}_test.csv', index=False)

processing protein 10 / 41
processing protein 20 / 41
processing protein 30 / 41
processing protein 40 / 41
processing protein 10 / 41
processing protein 20 / 41
processing protein 30 / 41
processing protein 40 / 41


In [1]:
prefix = 'rus_5_20'
import pandas as pd
df_valid = pd.read_csv(f'{prefix}_valid.csv')
df_test = pd.read_csv(f'{prefix}_test.csv'
                      )

In [2]:
df_valid

,protein_index,residue_index,target,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8,pred_9
0,0,0,0,-19.54246,-13.67023,-8.73524,-12.06873,-11.25087,-16.23873,-8.78210,-10.09841,-12.99963,-13.35773
1,0,1,0,-9.61697,-15.02601,-16.32852,-11.16953,-10.88497,-15.03135,-6.66089,-13.26890,-12.23049,-17.15543
2,0,2,0,-13.42243,-14.70558,-17.26652,-10.80779,-18.37910,-16.87887,-10.89012,-16.15878,-17.24353,-19.74384
3,0,3,0,-13.85699,-13.69305,-13.35709,-13.36703,-15.83232,-11.17222,-11.87955,-11.23246,-19.97917,-17.66171
4,0,4,0,-19.97662,-15.57662,-14.03138,-10.91142,-16.24856,-12.24010,-11.63122,-14.44032,-18.58406,-18.16607
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12798,40,278,0,-15.59839,-22.56648,-22.70077,-9.30785,-20.12647,-16.72729,-10.74963,-16.48566,-21.91821,-18.84735
12799,40,279,0,-10.51355,-6.56192,-9.05338,-5.09147,-11.75112,-16.03059,-4.20212,-9.02839,-13.03310,-8.18919
12800,40,280,0,-12.51012,-15.76716,-16.08012,-11.96154,-21.22735,-16.56955,-7.79483,-16.10415,-17.89726,-18.64878
12801,40,281,0,-20.18494,-15.23590,-19.51898,-6.73896,-20.01470,-20.73895,-11.16145,-17.93477,-18.54645,-17.90267


In [3]:
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, precision_score, matthews_corrcoef

def generate_mean_ensemble_metrics(df, threshold=0):
    sum_preds = df[list(filter(lambda a: a.startswith('pred_'), df.columns.tolist()))].sum(axis=1)
    final_prediction = (sum_preds > threshold).astype(int)

    # Sensitivity (Recall)
    sensitivity = recall_score(df['target'], final_prediction)

    # Specificity
    tn, fp, fn, tp = confusion_matrix(df['target'], final_prediction).ravel()
    specificity = tn / (tn + fp)

    # Accuracy
    accuracy = accuracy_score(df['target'], final_prediction)

    # Precision
    precision = precision_score(df['target'], final_prediction)
    mcc = matthews_corrcoef(df['target'], final_prediction)
    return {
        "sensitivity": sensitivity,
        "specificity": specificity,
        "accuracy": accuracy,
        "precision": precision,
        "mcc": mcc,
    }

generate_mean_ensemble_metrics(df_test, threshold=0)

{'sensitivity': 0.5677830940988836,
 'specificity': 0.9901559369283038,
 'accuracy': 0.9682801916405088,
 'precision': 0.7590618336886994,
 'mcc': 0.6407160506628494}

# Ensemble all of 10 models

MCC: 0.6477

In [6]:
from lib.utils import round_dict
import numpy as np
def generate_mean_ensemble_metrics_auto_threshold(df_valid, df_test=None, start=-10, end=10, step=0.1):
    thresholds = np.arange(start, end, step)
    mcc_values = []

    for threshold in thresholds:
        metrics = generate_mean_ensemble_metrics(df_valid, threshold=threshold)
        mcc_values.append(metrics['mcc'])
    
    best_threshold = thresholds[np.argmax(mcc_values)]

    test_metrics = generate_mean_ensemble_metrics(df_test, threshold=best_threshold) if df_test is not None else {}

    return round_dict({
        'best_threshold': best_threshold,
        'valid_mcc': np.max(mcc_values),
        **test_metrics
    }, 4)

generate_mean_ensemble_metrics_auto_threshold(df_valid, df_test)

{'best_threshold': 4.1,
 'valid_mcc': 0.6816,
 'sensitivity': 0.5582,
 'specificity': 0.9917,
 'accuracy': 0.9693,
 'precision': 0.7865,
 'mcc': 0.6477}

# Ensemble with 5 model of higher valid_mcc (Current method)

MCC: 0.6327

In [7]:
valid_mccs = []

for i in range(10):
    df_single_valid = df_valid[['target', f'pred_{i}']]
    df_single_test = df_test[['target', f'pred_{i}']]

    valid_mccs.append(generate_mean_ensemble_metrics_auto_threshold(df_single_valid)['valid_mcc'])


valid_mccs = np.array(valid_mccs)
valid_mccs

array([0.6435, 0.6216, 0.6401, 0.657 , 0.6322, 0.6398, 0.6257, 0.6386,
       0.6372, 0.6229])

In [33]:
top_5_indices = np.argsort(valid_mccs)[-5:]
top_5_indices

array([7, 5, 2, 0, 3])

In [39]:
df_highmcc_valid = df_valid[['target'] + ['pred_' + str(i) for i in top_5_indices]]
df_highmcc_test = df_test[['target'] + ['pred_' + str(i) for i in top_5_indices]]


generate_mean_ensemble_metrics_auto_threshold(df_highmcc_valid, df_highmcc_test)

{'best_threshold': -1.4,
 'valid_mcc': 0.6835,
 'sensitivity': 0.5901,
 'specificity': 0.9872,
 'accuracy': 0.9666,
 'precision': 0.7157,
 'mcc': 0.6327}

# Ensemble with 5 model set of maximal prediction difference

Does not work: this leads to lower mcc

In [8]:
from itertools import combinations
from scipy.spatial.distance import cosine
from scipy.special import expit

pairwise_diff = {}
thresholds = [generate_mean_ensemble_metrics_auto_threshold(df_valid[['target', f'pred_{i}']])['best_threshold'] for i in range(10)]
for i in range(10):
    for j in range(i+1, 10):
        pairwise_diff[(i, j)] = ((np.array(df_valid[f'pred_{i}']) > thresholds[i]) != (np.array(df_valid[f'pred_{j}']) > thresholds[j])).sum()
        pairwise_diff[(j, i)] = pairwise_diff[(i, j)]

In [9]:
pairwise_diff

{(0, 1): 251,
 (1, 0): 251,
 (0, 2): 218,
 (2, 0): 218,
 (0, 3): 232,
 (3, 0): 232,
 (0, 4): 250,
 (4, 0): 250,
 (0, 5): 242,
 (5, 0): 242,
 (0, 6): 206,
 (6, 0): 206,
 (0, 7): 227,
 (7, 0): 227,
 (0, 8): 240,
 (8, 0): 240,
 (0, 9): 244,
 (9, 0): 244,
 (1, 2): 205,
 (2, 1): 205,
 (1, 3): 221,
 (3, 1): 221,
 (1, 4): 279,
 (4, 1): 279,
 (1, 5): 241,
 (5, 1): 241,
 (1, 6): 243,
 (6, 1): 243,
 (1, 7): 236,
 (7, 1): 236,
 (1, 8): 199,
 (8, 1): 199,
 (1, 9): 241,
 (9, 1): 241,
 (2, 3): 208,
 (3, 2): 208,
 (2, 4): 252,
 (4, 2): 252,
 (2, 5): 232,
 (5, 2): 232,
 (2, 6): 214,
 (6, 2): 214,
 (2, 7): 233,
 (7, 2): 233,
 (2, 8): 202,
 (8, 2): 202,
 (2, 9): 248,
 (9, 2): 248,
 (3, 4): 270,
 (4, 3): 270,
 (3, 5): 246,
 (5, 3): 246,
 (3, 6): 228,
 (6, 3): 228,
 (3, 7): 227,
 (7, 3): 227,
 (3, 8): 212,
 (8, 3): 212,
 (3, 9): 246,
 (9, 3): 246,
 (4, 5): 276,
 (5, 4): 276,
 (4, 6): 262,
 (6, 4): 262,
 (4, 7): 281,
 (7, 4): 281,
 (4, 8): 272,
 (8, 4): 272,
 (4, 9): 306,
 (9, 4): 306,
 (5, 6): 238,
 (6, 5

In [10]:
from itertools import combinations
from scipy.special import comb as C
results = []
for comb in combinations(range(10), 5):
    diff_sum = 0
    for i in range(5):
        for j in range(i+1, 5):
            diff_sum += pairwise_diff[(comb[i], comb[j])]
    diff_sum /= C(5, 2)
    
    df_comb_valid = df_valid[['target'] + ['pred_' + str(i) for i in comb]]
    df_comb_test = df_test[['target'] + ['pred_' + str(i) for i in comb]]

    res = generate_mean_ensemble_metrics_auto_threshold(df_comb_valid, df_comb_test, start=-10, end=10, step=1)
    results.append((comb, diff_sum, res['valid_mcc'], res['mcc']))


results.sort(key=lambda x: x[1])


In [11]:
results[:10]

[((1, 2, 3, 6, 8), 215.0, 0.6692, 0.6274),
 ((0, 2, 3, 6, 8), 217.8, 0.6746, 0.6362),
 ((1, 2, 3, 7, 8), 218.4, 0.665, 0.6348),
 ((0, 1, 2, 3, 8), 218.8, 0.6686, 0.645),
 ((0, 1, 2, 6, 8), 219.6, 0.6741, 0.6405),
 ((1, 2, 3, 5, 8), 221.0, 0.6787, 0.6406),
 ((0, 1, 2, 3, 6), 222.6, 0.6758, 0.6324),
 ((2, 3, 6, 7, 8), 223.2, 0.6722, 0.6302),
 ((1, 2, 3, 8, 9), 223.4, 0.6678, 0.6367),
 ((1, 2, 5, 6, 8), 223.6, 0.6728, 0.6267)]

In [12]:
results[-10:]

[((3, 4, 6, 7, 9), 260.0, 0.6714, 0.629),
 ((1, 4, 5, 6, 9), 260.4, 0.6681, 0.6415),
 ((4, 6, 7, 8, 9), 261.2, 0.6761, 0.6285),
 ((1, 4, 6, 7, 9), 262.8, 0.6722, 0.6285),
 ((0, 4, 5, 7, 9), 263.6, 0.6806, 0.6395),
 ((2, 4, 5, 7, 9), 263.8, 0.6733, 0.6356),
 ((3, 4, 5, 7, 9), 266.2, 0.6719, 0.6325),
 ((1, 4, 5, 7, 9), 267.0, 0.6777, 0.6359),
 ((4, 5, 6, 7, 9), 267.4, 0.6802, 0.6355),
 ((4, 5, 7, 8, 9), 268.2, 0.6799, 0.6386)]

In [13]:
result_df = pd.DataFrame(results, columns=['combination', 'difference', 'valid_mcc', 'mcc'])

In [14]:
result_df

,combination,difference,valid_mcc,mcc
0,"(1, 2, 3, 6, 8)",215.0,0.6692,0.6274
1,"(0, 2, 3, 6, 8)",217.8,0.6746,0.6362
2,"(1, 2, 3, 7, 8)",218.4,0.6650,0.6348
3,"(0, 1, 2, 3, 8)",218.8,0.6686,0.6450
4,"(0, 1, 2, 6, 8)",219.6,0.6741,0.6405
...,...,...,...,...
247,"(2, 4, 5, 7, 9)",263.8,0.6733,0.6356
248,"(3, 4, 5, 7, 9)",266.2,0.6719,0.6325
249,"(1, 4, 5, 7, 9)",267.0,0.6777,0.6359
250,"(4, 5, 6, 7, 9)",267.4,0.6802,0.6355


In [18]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(result_df['difference'].values.reshape(-1,1), result_df['mcc'].values)

In [19]:
reg.coef_

array([3.51267265e-06])

In [20]:
reg.score(result_df['valid_mcc'].values.reshape(-1,1), result_df['mcc'].values)

-0.017854048910808773